In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import pickle
import mediapipe as mp

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_styled_landmarks(image, results):
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) 
    
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

def load_image(image_path):
    img=cv2.imread(image_path)
    img=cv2.resize(img,(480,480))
    image, results = mediapipe_detection(img, holistic)
    keypoints = extract_keypoints(results)
    return keypoints

model=load_model('model01.keras')


# 1. New detection variables
sequence = []
predictions = []
label=int(input("Enter the label number: "))
person_id=int(input("Enter Your id number: "))

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        
        frame=cv2.resize(frame,(480,480))
        
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-90:]
        
        if len(sequence) == 90:
            sequence=[sequence[int(j)] for j in np.arange(0,89,3)]
            sequence=np.array(sequence)
            np.save(f'New_Data\\{label}\\{person_id}.npy',sequence)
            break
            
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
with open('New_class.pkl', 'rb') as file:
    classes = pickle.load(file)

In [ ]:
classes[0]

In [ ]:
import os

# Define the range of directory names (0 to 210)
directory_names = range(98)

# Specify the parent directory where you want to create these subdirectories
parent_directory = "New_Data"

# Create directories
for directory_name in directory_names:
    # Join the parent directory path with the directory name
    directory_path = os.path.join(parent_directory, str(directory_name))
    
    # Check if the directory already exists, and create it if not
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

print("Directories created successfully.")

In [ ]:
import os
import numpy as np

# Set the root directory where your data folders are located
root_directory = "New_Data"

# Initialize empty lists to store data and labels
X = []
y = []

# Loop through each folder in the root directory
for class_folder in os.listdir(root_directory):
    class_folder_path = os.path.join(root_directory, class_folder)

    # Check if the path is a directory
    if os.path.isdir(class_folder_path):
        # Loop through each .npy file in the folder
        for file_name in os.listdir(class_folder_path):
            file_path = os.path.join(class_folder_path, file_name)

            # Check if the file is a .npy file
            if file_name.endswith(".npy"):
                # Load the data from the .npy file
                data = np.load(file_path)

                # Append data to X and corresponding label to y
                X.append(data)
                y.append(class_folder)

# Convert lists to NumPy arrays
X = np.array(X)
y = np.array(y)

# Print the shapes of X and y
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
